---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

## Data cleaning

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score

In [2]:
submit = 0
prefix = ""
if submit == 0:
    prefix = "readonly/"
    
train = pd.read_csv(prefix + 'train.csv', encoding = 'ISO-8859-1', low_memory = False)
test = pd.read_csv(prefix + 'test.csv')
addresses = pd.read_csv(prefix + 'addresses.csv')
latlon = pd.read_csv(prefix + 'latlons.csv')
train.drop(['payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status', 'compliance_detail'], 1, inplace=True)

In [358]:
train = train[np.isfinite(train['compliance'])]
train['compliance'] = train['compliance'].astype('Int64')

In [359]:
train = pd.merge(train, addresses, on ='ticket_id', how ='inner')
train = pd.merge(train, latlon, on ='address', how ='inner')

test = pd.merge(test, addresses, on ='ticket_id', how ='inner')
test = pd.merge(test, latlon, on ='address', how ='inner')

In [360]:
def data_clean(data):
#     data.dropna(subset=['violator_name', 'mailing_address_str_name', 'hearing_date'], inplace=True)
    data.drop(['city', 'state','country', 'mailing_address_str_number',
               'violation_zip_code', 'non_us_str_code', 'grafitti_status'], 1, inplace=True)
    data['lat'].fillna((data['lat'].mean()), inplace=True)
    data['lon'].fillna((data['lon'].mean()), inplace=True)
    
    return data

In [361]:
def create_simple(data):
    data = data.select_dtypes(exclude=['object'])
    
    return data

In [362]:
def one_hot_encoding(data, col1, col2):
    categorical_columns = [col1, col2]
    df = pd.get_dummies(data, columns=categorical_columns)
    
    return df

In [363]:
def label_encoder(data, col, col_name, le=None):
    # If encoder was not provided
    if not le:
        le = preprocessing.LabelEncoder()
    le_input = data[col].tolist()
    le.fit(le_input)
    le_output = le.transform(le_input)
    data[col_name] = le_output
    data.drop(col, axis=1, inplace=True)
    
    return data, le

In [364]:
def diff_days(data, col1, col2):
    col1_to_date = pd.to_datetime(data[col1])
    col2_to_date = pd.to_datetime(data[col2])
    diff = (col2_to_date - col1_to_date).dt.days
    data['diff_days'] = diff
    
    return data

In [365]:
train_clean = data_clean(train)
train_clean.dropna(subset=['violator_name', 'mailing_address_str_name', 'hearing_date'], inplace=True)
test_clean = data_clean(test)

In [366]:
train_clean_ohe = one_hot_encoding(train_clean, 'agency_name', 'disposition')
test_clean_ohe = one_hot_encoding(test_clean, 'agency_name', 'disposition')

In [367]:
train_clean_le, le = label_encoder(train_clean_ohe, 'zip_code', 'zip_code_le')
test_clean_le, _ = label_encoder(test_clean_ohe, 'zip_code', 'zip_code_le', le)

train_clean_le, le = label_encoder(train_clean_le, 'violation_code', 'violation_code_le')
test_clean_le, _ = label_encoder(test_clean_le, 'violation_code', 'violation_code_le', le)

In [368]:
train_clean_diff = diff_days(train_clean_le, 'ticket_issued_date', 'hearing_date')
test_clean_diff = diff_days(test_clean_le, 'ticket_issued_date', 'hearing_date')

In [369]:
train_simple = create_simple(train_clean_diff)
test_simple = create_simple(test_clean_diff)

In [370]:
train_simple.loc[:, 'disposition_Responsible (Fine Waived) by Admis'] = '0'
train_simple.loc[:, 'disposition_Responsible - Compl/Adj by Default'] = '0'
train_simple.loc[:, 'disposition_Responsible - Compl/Adj by Determi'] = '0'
train_simple.loc[:, 'disposition_Responsible by Dismissal'] = '0'
test_simple.loc[:, 'agency_name_Health Department'] = '0'
train_simple.drop('agency_name_Neighborhood City Halls', 1, inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [371]:
train_simple.drop('ticket_id', 1, inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [378]:
test_simple.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61001 entries, 0 to 61000
Data columns (total 26 columns):
ticket_id                                                     61001 non-null int64
violation_street_number                                       61001 non-null float64
fine_amount                                                   61001 non-null float64
admin_fee                                                     61001 non-null float64
state_fee                                                     61001 non-null float64
late_fee                                                      61001 non-null float64
discount_amount                                               61001 non-null float64
clean_up_cost                                                 61001 non-null float64
judgment_amount                                               61001 non-null float64
lat                                                           61001 non-null float64
lon                                        

# PCA with normalization

In [239]:
# from sklearn import preprocessing
# my_best_df = pd.DataFrame()
# my_best_df_cat = pd.DataFrame()
# for col in train:
#     if type(train[col][0]) == np.float64:
#         my_best_df[col] = train[col]
#     else:
#         my_best_df_cat[col] = train[col]
# names = my_best_df.columns
# minMaxScaler = preprocessing.MinMaxScaler()
# my_best_df_norm = minMaxScaler.fit_transform(my_best_df)
# scaled_df = pd.DataFrame(my_best_df_norm, columns=names)
# train_norm = pd.concat([scaled_df, my_best_df_cat], axis=1)
# train_norm.head(3)

In [240]:
# train_norm = train_norm[['violation_street_number', 'fine_amount',
#        'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
#        'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
#        'lat', 'lon', 'disposition']]

In [241]:
# train_norm_encode = pd.get_dummies(train_norm, columns=['disposition'])
# train_norm_encode = train_norm_encode.dropna()

# disposition = pd.get_dummies(my_best_df_cat['disposition'])

In [242]:
# train_norm_encode.drop('disposition_Responsible by Determination', 1, inplace=True)

In [243]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=3)
# train_pca = pca.fit_transform(train_norm_encode)

In [244]:
# print(pca.explained_variance_ratio_)

In [245]:
# train_df = pd.DataFrame(data=train_pca, columns=['principal_com_1', 'principal_com_2', 'principal_com_3'])

In [246]:
# test = test.drop(['violation_zip_code', 'mailing_address_str_number', 'non_us_str_code'], 1)
# test['lat'].fillna((test['lat'].mean()), inplace=True)
# test['lon'].fillna((test['lon'].mean()), inplace=True)

In [247]:
# from sklearn import preprocessing
# my_best_df_test = pd.DataFrame()
# my_best_df_cat_test = pd.DataFrame()
# for col in test:
#     if type(test[col][0]) == np.float64:
#         my_best_df[col] = test[col]
#     else:
#         my_best_df_cat_test[col] = test[col]
# names = my_best_df_test.columns
# minMaxScaler = preprocessing.MinMaxScaler()
# my_best_df_norm_test = minMaxScaler.fit_transform(my_best_df_test)
# scaled_df_test = pd.DataFrame(my_best_df_norm_test, columns=names)
# train_norm_test = pd.concat([scaled_df_test, my_best_df_test], axis=1)
# train_norm_test.head(3)

# Baseline model - dummy classifier

In [248]:
# from sklearn.dummy import DummyClassifier

# dummy_clf = DummyClassifier(strategy='most_frequent')
# dummy_clf.fit(X_train, y_train)

# dummy_clf.predict(X_test)
# dummy_clf.score(X_test, y_test)

# Random forest model 

In [374]:
from sklearn.model_selection import train_test_split

X = train_simple.loc[:, train_simple.columns != 'compliance']
y = train_simple['compliance']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [375]:
print(X_train.shape, X_test.shape)

(119718, 25) (39906, 25)


In [376]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_features = 25, random_state = 0, max_depth = 9, min_samples_leaf= 3, n_estimators = 10)
clf.fit(X_train, y_train)
print('Accuracy of RF classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of RF classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of RF classifier on training set: 0.95
Accuracy of RF classifier on test set: 0.94


In [379]:
prediction_val_data = clf.predict_proba(np.array(test_simple))
answer = pd.Series(prediction_val_data[:,1], index = test_simple['ticket_id'].tolist())
roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])

In [ ]:
def blight_model():
    
    return answer

# Logistic regression

In [34]:
# features_corr = train.corr()

In [262]:
# fig, ax = plt.subplots(figsize=(10,10))
# sns.heatmap(features_corr, fmt = ".2f", cmap="YlGnBu", square = True,
#             linewidths = 0.5, annot=True, cbar_kws = {"shrink": .815})
# plt.show()

In [26]:
# X = train.drop(['compliance'], axis = 1)
# y = train['compliance']

In [28]:
# import sklearn as sk
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y)

In [263]:
# from sklearn.linear_model import LogisticRegression

# logisticregression_model = LogisticRegression(max_iter = 1000)
# logisticregression_model.fit(X_train, y_train)

In [32]:
# train_acc_logreg = logisticregression_model.score(X_train, y_train)

In [34]:
# prediction = logisticregression_model.predict(X_test)